In [45]:
! pip install Pillow


In [46]:
import arabic_reshaper
from bidi.algorithm import get_display
import numpy as np
from PIL import ImageFont, ImageDraw, Image

In [47]:
import cv2
import numpy as np


class Extract_Characters:
    def extractCharacters(self, img):
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret2, binary_img = cv2.threshold(
            gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
        )
        binary_img = ~binary_img
        kernel = np.ones((3, 3), np.uint8)
        nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(
            binary_img, connectivity=4
        )
        resized_character = []
        for i in range(len(centroids)):
            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            if h <= 80 and h >= 20 and w >= 15 and w < 30:
                source = binary_img[y - 10 : y + h + 10, x : x + w]
                source = cv2.resize(source, (32, 32))
                source = cv2.resize(source, (16, 16))
                source = cv2.copyMakeBorder(source, 8, 8, 8, 8, 0)
                resized_character.append((source, x))
        resized_character = sorted(resized_character, key=lambda x: x[1])
        return resized_character

    def extract(self, original_img):
        resized_img = cv2.resize(original_img, (200, 150))
        resized_num_character = self.extractCharacters(resized_img[:, 0:100])
        resized_char_character = self.extractCharacters(resized_img[:, 100:])
        return [x[0] for x in resized_num_character], [
            x[0] for x in resized_char_character
        ]


import cv2
import numpy as np
from keras.models import model_from_json


class Character_Recognizer:
    def __init__(self):
        self.arabic_characters = [
            "ا",
            "ب",
            "ت",
            "ث",
            "ج",
            "ح",
            "خ",
            "د",
            "ذ",
            "ر",
            "ز",
            "س",
            "ش",
            "ص",
            "ض",
            "ط",
            "ظ",
            "ع",
            "غ",
            "ف",
            "ق",
            "ك",
            "ل",
            "م",
            "ن",
            "ه",
            "و",
            "ي",
        ]
        # self.arabic_characters = ['alf', 'beh', 'teh', 'theh', 'gem', 'hah', 'khah', 'dal', 'zal',
        #                           'reh', 'zen', 'sen', 'shen', 'sad', 'daad', 'tah', 'zah', 'een',
        #                           'gheen', 'feh', 'qaaf', 'kaf', 'lam', 'mem', 'noon', 'heeh', 'waw', 'yeh']
        json_file = open("Characters Model/character model json.json", "r")
        loaded_model_json = json_file.read()
        json_file.close()
        self.loaded_model = model_from_json(loaded_model_json)
        # load weights into new model
        self.loaded_model.load_weights("Characters Model/character weights.h5")

    def get_sides(self, length):
        if length % 2 == 0:
            return length // 2, length // 2
        else:
            return (length - 1) // 2, 1 + (length - 1) // 2

    def preprocess(self, character):
        (wt, ht) = (32, 32)
        (h, w) = character.shape
        fx = w / wt
        fy = h / ht
        f = max(fx, fy)
        newSize = (max(min(wt, int(w / f)), 1), max(min(ht, int(h / f)), 1))
        character = cv2.resize(character, newSize)

        if character.shape[0] < 32:
            add_zeros_up = np.zeros(
                (self.get_sides(32 - character.shape[0])[0], character.shape[1])
            )
            add_zeros_down = np.zeros(
                (self.get_sides(32 - character.shape[0])[1], character.shape[1])
            )
            character = np.concatenate((add_zeros_up, character))
            character = np.concatenate((character, add_zeros_down))

        if character.shape[1] < 32:
            add_zeros_left = np.zeros((32, self.get_sides(32 - character.shape[1])[0]))
            add_zeros_right = np.zeros((32, self.get_sides(32 - character.shape[1])[1]))

            character = np.concatenate((add_zeros_left, character), axis=1)
            character = np.concatenate((character, add_zeros_right), axis=1)

        character = character.T / 255.0
        character = np.expand_dims(character, axis=2)
        return character

    def ocr(self, img):
        # img  = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        img = self.preprocess(img)
        img = img.reshape(-1, 32, 32, 1)
        pred = self.loaded_model.predict([[img]])
        return self.arabic_characters[np.argmax(pred)]


import cv2
import numpy as np


class Car_Plate_Detection:
    def __init__(self):
        modelConfiguration = "CarPlateModel/yolov3-tiny.cfg"
        modelWeights = "CarPlateModel/yolov3-tiny.backup"
        self.net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
        self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    def getOutputsNames(self, n):
        layersNames = n.getLayerNames()
        return [layersNames[i - 1] for i in n.getUnconnectedOutLayers()]

    def postprocess(self, fr, outs, confT, nmsT):
        frameHeight = fr.shape[0]
        frameWidth = fr.shape[1]
        classIds = []
        confidences = []
        boxes = []
        for o in outs:
            for detection in o:
                scores = detection[5:]
                classId = np.argmax(scores)
                confidence = scores[classId]
                if confidence > confT:
                    center_x = int(detection[0] * frameWidth)
                    center_y = int(detection[1] * frameHeight)
                    width = int(detection[2] * frameWidth)
                    height = int(detection[3] * frameHeight)
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)
                    classIds.append(classId)
                    confidences.append(float(confidence))
                    boxes.append([left, top, width, height])

        indices = cv2.dnn.NMSBoxes(boxes, confidences, confT, nmsT)
        cropped = None
        for i in indices:
            box = boxes[i]
            left = max(box[0], 0)
            top = max(box[1], 0)
            width = max(box[2], 0)
            height = max(box[3], 0)
            if height > width:
                continue
            cropped = fr[top : (top + height), left : (left + width)]

        return len(indices) > 0, cropped

    def Detect_Plate(self, frame):
        blob = cv2.dnn.blobFromImage(
            frame, 1 / 255, (416, 416), [0, 0, 0], 1, crop=False
        )
        self.net.setInput(blob)
        run = self.net.forward(self.getOutputsNames(self.net))
        rec, plateImg = self.postprocess(frame, run, 0.5, 0.5)
        x = np.array([2])
        if rec > 0 and type(plateImg) == type(x) and plateImg.all() is not None:
            cv2.imwrite("Plates From Model/0.png", plateImg.astype(np.uint8))
            return plateImg

        return False


import cv2
import numpy as np
import keras


class Number_Recognizer:
    def __init__(self):
        self.arabic_digit = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
        json_file = open("Characters Model/digits model json.json", "r")
        loaded_model_json = json_file.read()
        json_file.close()
        self.loaded_model = keras.models.model_from_json(loaded_model_json)
        self.loaded_model.load_weights("Characters Model/digits weights.h5")

    def get_sides(self, length):
        if length % 2 == 0:
            return length // 2, length // 2
        else:
            return (length - 1) // 2, 1 + (length - 1) // 2

    def preprocess(self, character):
        (wt, ht) = (28, 28)
        (h, w) = character.shape
        fx = w / wt
        fy = h / ht
        f = max(fx, fy)
        newSize = (max(min(wt, int(w / f)), 1), max(min(ht, int(h / f)), 1))
        character = cv2.resize(character, newSize)
        if character.shape[0] < 28:
            add_zeros_up = np.zeros(
                (self.get_sides(28 - character.shape[0])[0], character.shape[1])
            )
            add_zeros_down = np.zeros(
                (self.get_sides(28 - character.shape[0])[1], character.shape[1])
            )
            character = np.concatenate((add_zeros_up, character))
            character = np.concatenate((character, add_zeros_down))
        if character.shape[1] < 28:
            add_zeros_left = np.zeros((28, self.get_sides(28 - character.shape[1])[0]))
            add_zeros_right = np.zeros((28, self.get_sides(28 - character.shape[1])[1]))

            character = np.concatenate((add_zeros_left, character), axis=1)
            character = np.concatenate((character, add_zeros_right), axis=1)
        character = character.T / 255.0
        character = np.expand_dims(character, axis=2)
        return character

    def ocr(self, img):
        img = self.preprocess(img)
        img = img.reshape(-1, 28, 28, 1)
        pred = self.loaded_model.predict(img)
        return self.arabic_digit[np.argmax(pred)]


cr = Character_Recognizer()
nr = Number_Recognizer()
Ec = Extract_Characters()
cp = Car_Plate_Detection()

# Change Path to Picture
image = cv2.imread("Test/1.png")


PlateImg = cp.Detect_Plate(image)
if PlateImg.all() == None:
    print("No plate found in image")
else:
    numbers, characters = Ec.extract(PlateImg)
    word = []
    for i in range(len(numbers)):
        word.append(nr.ocr(numbers[i]))

    for i in range(len(characters)):
        word.append(cr.ocr(characters[i]))

    image = cv2.resize(image, (480, 480))
    cv2.putText(
        image,
        str(word),
        (50, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (120, 120, 0),
        2,
        cv2.LINE_AA,
    )
    cv2.imwrite("Output/5.png", image)
    cv2.imshow("Photo", image)
    k = cv2.waitKey(5000)
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step


QObject::moveToThread: Current thread (0xde5ac20) is not the object's thread (0xdeaed90).
Cannot move to target thread (0xde5ac20)

QObject::moveToThread: Current thread (0xde5ac20) is not the object's thread (0xdeaed90).
Cannot move to target thread (0xde5ac20)

QObject::moveToThread: Current thread (0xde5ac20) is not the object's thread (0xdeaed90).
Cannot move to target thread (0xde5ac20)

QObject::moveToThread: Current thread (0xde5ac20) is not the object's thread (0xdeaed90).
Cannot move to target thread (0xde5ac20)

QObject::moveToThread: Current thread (0xde5ac20) is not the object's thread (0xdeaed90).
Cannot move to target thread (0xde5ac20)

QObject::moveToThread: Current thread (0xde5ac20) is not the object's thread (0xdeaed90).
Cannot move to target thread (0xde5ac20)

QObject::moveToThread: Current thread (0xde5ac20) is not the object's thread (0xdeaed90).
Cannot move to target thread (0xde5ac20)

QObject::moveToThread: Current thread (0xde5ac20) is not the object's thread